<a href="https://colab.research.google.com/github/nguyenkimthach/Optimization_Binary_Neural_Network/blob/main/BinaryNet_MnistFashion_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BinaryNet on Fashion MNIST

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!ls /content/drive/MyDrive
#dataset_path = '/content/drive/MyDrive/newdataset/

save_path='/content/drive/MyDrive/DATASET2/BinaryNet_mnistFashion.h5'

In [ ]:
!pip install tensorflow==2.7.1

In [ ]:
!pip install pyyaml h5py
!pip install larq

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import larq as lq
import numpy as np
import time
# For ploting
import matplotlib.pyplot as plt

### Download and prepare the MNIST dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

In [ ]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantConv2D(16, (3, 3),
                                kernel_quantizer="ste_sign",
                                kernel_constraint="weight_clip",
                                use_bias=False,
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantConv2D(16, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(32, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(32, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

### Compile and train the model

Note: This may take a few minutes depending on your system.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
start_time = time.time()
trained_model=model.fit(
    train_images,
    train_labels,
    batch_size=64,
    epochs=10,
    validation_data=(test_images, test_labels),
    shuffle=True
)
#print("execution time:  %s ms" % ((time.time() - start_time)*1000))


test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
for layer in model.layers:
    print(layer.name, layer)
#save mode
model.save(save_path)

### Evaluate the model

In [ ]:
plt.plot(trained_model.history['accuracy'])
#plt.plot(trained_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.max(trained_model.history['accuracy']))
#print(np.min(trained_model.history['val_accuracy']))

In [ ]:
plt.plot(trained_model.history['loss'])
#plt.plot(trained_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.min(trained_model.history['loss']))
#print(np.min(trained_model.history['val_loss']))

In [ ]:
for layer in model.layers:
    print(layer.name, layer)
#save mode
model.save(save_path)

In [ ]:
print(f"Test accuracy {test_acc * 100:.2f} %")

In [ ]:
prediction = model.predict(test_images[:2])
print("prediction shape:", prediction.shape)